# NRM python upstream client library tutorial

This tutorial covers the use of NRM's python upstream client library, in the context of running an external resource management strategy. Its cell's output are deterministic, and the executed version that is vendored in the source tree is checked by the project's CI, so its behavior should always be up-to-date with the latest version of the software. No cells should be throwing exceptions, as  

This notebook uses `nrm`'s python library bindings and needs the `nrmd` daemon in the `$PATH`. Assuming the project is cloned (and the code unmodified), one therefore needs to run the following from the root of the project before running it:

```bash
./shake.sh build
./shake.sh pyclient 
```

The next cell sets the working directory of the notebook at the root of the project.

In [1]:
%%capture
cd ..

The next cell imports the upstream client library. and configures hosts.

In [2]:
from nrm.upstreamclient import Local,Remote

This notebook will start `nrmd` on the same machine as the notebok, but the same interface should be available for remote execution:

In [3]:
host=Local()
#host=Remote( target="cc@129.114.108.201")

Note that the two classes offer the same method to start nrmd and interact via blocking message passing primitives. The following cell should show their docstrings and code when executed interactively.

In [4]:
# Local??
# Remote??

The next cell defines some dode (`nrmd`) daemon configurations as dictionaries:

In [5]:
daemonCfgs = {
    'redirected_log': { "logfile" : "/tmp/logfile_experiment1"},
    #"other":'todo'
}

A workload need a command, some arguments, and a manifest, also represented as a python dictionary.

In [6]:
workloads = {
    "dummy":{ "cmd":"sleep",
              "args": ["333"] ,
              "sliceID": "toto",
              "manifest":{"app": {
                              "slice": {
                                  "cpus": 1,
                                  "mems": 1 } 
                                       },
                          "name": "default"
                         },
            },
    #"other":todo
}

Manifest options are documented in schema file [resources/manifestSchema.json](../resources/manifestSchema.json) . Clicking this link should open a useful tab - modern web browsers have JSON data explorers. Example manifest files are in [resources/examples](../resources/examples) in JSON/YAML/Dhall format. For instance, the manifest file [resources/examples/perfwrap.json](../resources/examples/perfwrap.json) enables enables performance monitoring:

In [7]:
cat resources/examples/perfwrap.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapper": {
        "perfLimit": 100000,
        "perfFreq": 1
      }
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


Remember that those are one little step away from being Python dictionaries:

In [8]:
import json
with open("resources/examples/perfwrap.json") as f: print(json.load(f))

{'hwbind': False, 'app': {'scheduler': {'fIFO': {}}, 'power': {'slowdown': 1, 'profile': False, 'policy': 'noPowerPolicy'}, 'perfwrapper': {'perfwrapper': {'perfLimit': 100000, 'perfFreq': 1}}, 'slice': {'cpus': 1, 'mems': 1}}, 'name': 'default'}


Under-specified manifests like the one in our `workloads` above (with missing optional fields from the schema) fill missing values with defaults, which can be seen in file [resources/examples/default.json](../../resources/examples/default.json):

In [9]:
cat resources/examples/default.json | jq

{
  "hwbind": false,
  "app": {
    "scheduler": {
      "fIFO": {}
    },
    "power": {
      "slowdown": 1,
      "profile": false,
      "policy": "noPowerPolicy"
    },
    "perfwrapper": {
      "perfwrapperDisabled": {}
    },
    "slice": {
      "cpus": 1,
      "mems": 1
    }
  },
  "name": "default"
}


The next cell defines some experiments:

In [10]:
experiments = { "example": (daemonCfgs['redirected_log'], workloads["dummy"]),
                #"other": todo
              }

The next two cells show how to start and stop the daemon. A failure in either of them indicates a problem with NRM's setup.

In [11]:
host.start_daemon(daemonCfgs['redirected_log'])
assert(host.check_daemon())

In [12]:
host.stop_daemon()
assert(host.check_daemon() == False)

We now are ready to run an external resource management strategy. Using the low-level message passing interface: